## Interactively running scenaria

In [1]:
import pandas as pd
import os
import pickle

from pipeline.scenario import Scenario
from pipeline.constants import *

In [2]:
scenario_params_grid = [
                        dict(
                            **dates,
                            vaccine_effectiveness=vaccine_effectiveness,
                            daily_vaccine_budget=daily_vaccine_budget,
                            min_allocation_factor=min_allocation_factor,
                            balanced_location=balanced_location
                        )
                        for dates in DATES_GRID
                        for vaccine_effectiveness in VACCINE_EFFECTIVENESS_GRID
                        for daily_vaccine_budget in DAILY_VACCINE_BUDGET_GRID
                        for min_allocation_factor in MIN_ALLOCATION_FACTOR_GRID
                        for balanced_location in BALANCED_LOCATIONS_GRID
                    ]

In [3]:
for i, scenario_params in enumerate(scenario_params_grid):
    
    print("Scenario",i)

    start_date = scenario_params["start_date"]
    end_date = scenario_params["start_date"]
    mortality_rate_path = f"{MORTALITY_RATES_PATH}2021-base.npy"
    reload_mortality_rate = os.path.isfile(mortality_rate_path)

    baseline_obj_val, optimized_obj_val = Scenario(**scenario_params).run(
        model_path=f"{MODEL_PATH_PATH}{i}.pickle",
        baseline_solution_path=f"{BASELINE_SOLUTION_PATH}{i}.pickle",
        optimized_solution_path=f"{OPTIMIZED_SOLUTION_ATH}{i}.pickle",
        mortality_rate_path=mortality_rate_path,
        reload_mortality_rate=reload_mortality_rate
    )
    scenario_params["baseline_obj_val"] = baseline_obj_val
    scenario_params["optimized_obj_val"] = optimized_obj_val

results = pd.DataFrame(scenario_params_grid)
results["abs_improvement"] = results["baseline_obj_val"] - results["optimized_obj_val"]
results["pct_improvement"] = results["abs_improvement"] / results["baseline_obj_val"] * 1e2
results.to_csv(RESULTS_PATH)

Scenario 0
Loading model...
Running baseline...
Optimizing...
Restart: 1/1
Iteration: 0/10 	 Objective value: 54314.00
Academic license - for non-commercial use only - expires 2021-04-02
Using license file /Users/vvdig/gurobi.lic
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 304963 rows, 301347 columns and 1236173 nonzeros
Model fingerprint: 0x3007a571
Variable types: 300796 continuous, 551 integer (500 binary)
Coefficient statistics:
  Matrix range     [3e-09, 6e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 203824 rows and 198621 colu

Iteration: 2/10 	 Objective value: 49192.15
No improvement found in 1 iterations - terminating search for trial
Post-processing solution
Objective value after post-processing: 49027.18


In [4]:
population = pd.read_csv(POPULATION_DATA_PATH)
states = population['state'].unique()

baseline_solutions = []
optimized_solutions = []

for i in range(len(scenario_params_grid)):
    baseline_solution_path=f"{BASELINE_SOLUTION_PATH}{i}.pickle"
    baseline_solutions.append(pickle.load(open(baseline_solution_path, "rb")))
    optimized_solution_path=f"{OPTIMIZED_SOLUTION_ATH}{i}.pickle"
    optimized_solutions.append(pickle.load(open(optimized_solution_path, "rb")))
    print("\n=====")
    print("Scenario", i)
    print([(optimized_solutions[i].locations[j], states[j]) for j in range(len(states))])
    print(optimized_solutions[i].get_total_deaths(), baseline_solutions[i].get_total_deaths())
    


=====
Scenario 0
[(1.0, 'Alabama'), (1.0, 'Alaska'), (2.0, 'Arizona'), (2.0, 'Arkansas'), (2.0, 'California'), (5.0, 'Colorado'), (2.0, 'Connecticut'), (1.0, 'Delaware'), (1.0, 'District of Columbia'), (7.0, 'Florida'), (1.0, 'Georgia'), (1.0, 'Hawaii'), (1.0, 'Idaho'), (1.0, 'Illinois'), (1.0, 'Indiana'), (1.0, 'Iowa'), (4.0, 'Kansas'), (2.0, 'Kentucky'), (1.0, 'Louisiana'), (1.0, 'Maine'), (1.0, 'Maryland'), (1.0, 'Massachusetts'), (1.0, 'Michigan'), (1.0, 'Minnesota'), (2.0, 'Mississippi'), (2.0, 'Missouri'), (1.0, 'Montana'), (2.0, 'Nebraska'), (1.0, 'Nevada'), (2.0, 'New Hampshire'), (2.0, 'New Jersey'), (2.0, 'New Mexico'), (6.0, 'New York'), (1.0, 'North Carolina'), (1.0, 'North Dakota'), (2.0, 'Ohio'), (2.0, 'Oklahoma'), (1.0, 'Oregon'), (2.0, 'Pennsylvania'), (1.0, 'Rhode Island'), (2.0, 'South Carolina'), (1.0, 'South Dakota'), (1.0, 'Tennessee'), (8.0, 'Texas'), (3.0, 'Utah'), (1.0, 'Vermont'), (3.0, 'Virginia'), (6.0, 'Washington'), (1.0, 'West Virginia'), (1.0, 'Wisconsin

In [5]:
V = optimized_solutions[0].vaccinated
print(np.nansum(V), V.shape[2]*0.6e6)

for t in range(V.shape[2]):
    print(V[:, :, t].sum())

48701979.785361 54600000.0
600000.0000000001
599706.6510723213
600000.0
599908.7133387598
583934.5146463108
591572.5621312908
599999.9999999999
541534.0649986217
559766.6078267789
550994.8252371682
501719.0505640721
579234.1125543767
541117.4074705652
545110.2452545362
515671.9557906912
535424.7905897027
587782.0880179416
591298.5285679334
417487.87236756727
577607.7380633571
536390.8138998279
589026.350516034
593351.2882102992
597250.2192321122
600000.0
497389.273412434
382362.12303167797
599999.9999999999
583121.1899990304
587683.3389444347
570470.4552627592
452442.13247621286
481685.9426413332
560492.8312468678
582195.0152248282
600000.0
549432.6619044254
452564.7125646656
531836.6162169365
582664.9428848238
552766.5121353935
564402.266727875
580970.8323352525
585993.5564115196
433185.899751273
508250.04803935956
600000.0
557768.8448708195
549626.9824516113
580425.9671869462
574955.3884332146
546632.18358605
449815.3747661145
522169.2436713743
514822.368388833
438608.01666264236
482